Having imputed the csv in df_impute.ipynb, I now perform text preprocessing.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("imputed_fmt_28-10-21.csv")
df.head()

,title,url,date
0,Why remove gender discrimination reforms from ...,https://www.freemalaysiatoday.com/category/nat...,2021-10-29
1,"6,060 new Covid-19 cases reported",https://www.freemalaysiatoday.com/category/nat...,2021-10-29
2,"Don’t execute mentally disabled Malaysian, rig...",https://www.freemalaysiatoday.com/category/nat...,2021-10-29
3,Bar allowed to be ‘friends of the court’ in Na...,https://www.freemalaysiatoday.com/category/nat...,2021-10-29
4,"Court fines retiree RM22,000 for failure to re...",https://www.freemalaysiatoday.com/category/nat...,2021-10-29


First I remove all commas, because figures such as 1,000 are common in news headlines. Then, I tokenize the headlines and remove stop words.

In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import string

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.replace(',', '')
    text = " ".join(re.split('\W+', text))
    word_tokens = word_tokenize(text)


    filtered_sentence = []

    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    filtered_sentence = [item for item in filtered_sentence if not item.isdigit()]
            
    return (", ".join(filtered_sentence))

print(clean_text("This is a sample sentence, showing off the stop words filtration."))

sample, sentence, showing, stop, words, filtration


In [4]:
df['cleaned'] = df['title'].apply(clean_text)
df.head()

,title,url,date,cleaned
0,Why remove gender discrimination reforms from ...,https://www.freemalaysiatoday.com/category/nat...,2021-10-29,"remove, gender, discrimination, reforms, Emplo..."
1,"6,060 new Covid-19 cases reported",https://www.freemalaysiatoday.com/category/nat...,2021-10-29,"new, Covid, cases, reported"
2,"Don’t execute mentally disabled Malaysian, rig...",https://www.freemalaysiatoday.com/category/nat...,2021-10-29,"execute, mentally, disabled, Malaysian, rights..."
3,Bar allowed to be ‘friends of the court’ in Na...,https://www.freemalaysiatoday.com/category/nat...,2021-10-29,"Bar, allowed, friends, court, Najib, son, tax,..."
4,"Court fines retiree RM22,000 for failure to re...",https://www.freemalaysiatoday.com/category/nat...,2021-10-29,"Court, fines, retiree, RM22000, failure, repor..."


In [5]:
df = df.drop(['title', 'url'], axis = 1)
df.head()

,date,cleaned
0,2021-10-29,"remove, gender, discrimination, reforms, Emplo..."
1,2021-10-29,"new, Covid, cases, reported"
2,2021-10-29,"execute, mentally, disabled, Malaysian, rights..."
3,2021-10-29,"Bar, allowed, friends, court, Najib, son, tax,..."
4,2021-10-29,"Court, fines, retiree, RM22000, failure, repor..."


In [6]:
df = df.groupby(['date'])['cleaned'].apply(lambda x: ','.join(x)).reset_index()
df['cleaned'] = df['cleaned'].apply(lambda x: x.split(","))
df.head()

,date,cleaned
0,2019-12-31,"[PIA, special, flight, brings, home, hund..."
1,2020-01-01,"[caught, smoking, eateries, first, day, e..."
2,2020-01-02,"[Petition, support, Maszlee, gets, signatu..."
3,2020-01-03,"[one, recognises, Mavcom, civil, aviation,..."
4,2020-01-04,"[PH, discuss, PM, handover, says, Syed, ..."


In [7]:
df.shape

(664, 2)

In [8]:
df.drop(index=df.index[0], 
        axis=0, 
        inplace=True)
df.head()

,date,cleaned
1,2020-01-01,"[caught, smoking, eateries, first, day, e..."
2,2020-01-02,"[Petition, support, Maszlee, gets, signatu..."
3,2020-01-03,"[one, recognises, Mavcom, civil, aviation,..."
4,2020-01-04,"[PH, discuss, PM, handover, says, Syed, ..."
5,2020-01-05,"[Umno, make, constitutional, proposal, say..."


I want to create a word cloud, so I count the number of tokens. Then, I save to json.

In [9]:
from collections import Counter

df['counter'] = df['cleaned'].apply(lambda x: dict(Counter(x)))
df.head()

,date,cleaned,counter
1,2020-01-01,"[caught, smoking, eateries, first, day, e...","{'caught': 1, ' smoking': 1, ' eateries': 1, '..."
2,2020-01-02,"[Petition, support, Maszlee, gets, signatu...","{'Petition': 1, ' support': 1, ' Maszlee': 3, ..."
3,2020-01-03,"[one, recognises, Mavcom, civil, aviation,...","{'one': 1, ' recognises': 1, ' Mavcom': 1, ' c..."
4,2020-01-04,"[PH, discuss, PM, handover, says, Syed, ...","{'PH': 1, ' discuss': 1, ' PM': 2, ' handover'..."
5,2020-01-05,"[Umno, make, constitutional, proposal, say...","{'Umno': 1, ' make': 1, ' constitutional': 1, ..."


In [10]:
df = df.drop(['cleaned'], axis = 1)
df.head()

,date,counter
1,2020-01-01,"{'caught': 1, ' smoking': 1, ' eateries': 1, '..."
2,2020-01-02,"{'Petition': 1, ' support': 1, ' Maszlee': 3, ..."
3,2020-01-03,"{'one': 1, ' recognises': 1, ' Mavcom': 1, ' c..."
4,2020-01-04,"{'PH': 1, ' discuss': 1, ' PM': 2, ' handover'..."
5,2020-01-05,"{'Umno': 1, ' make': 1, ' constitutional': 1, ..."


In [11]:
df.to_json("preprocessed_fmt.json", orient='records')